### Setup


In [ ]:
!pip install imageio > /dev/null 2>&1

In [ ]:
from google.colab import drive
PATH_TO_ENVIRONMENT = '/content/drive/MyDrive/Master_Thesis/Final/Predator-Prey_Environment'
PATH_TO_ALGORITHMS = '/content/drive/MyDrive/Master_Thesis/Final/Algorithm'

PATH_TO_PREY_MODEL = '/content/drive/MyDrive/Master_Thesis/Final/Algorithm/Prey.pth'
PATH_TO_PREDATOR_MODEL = '/content/drive/MyDrive/Master_Thesis/Final/Algorithm/Predator.pth'

drive.mount('/content/drive')

!cp {PATH_TO_ALGORITHMS}/colab_helpers.py /content
!cp {PATH_TO_ALGORITHMS}/A2C.py /content

!cp -r {PATH_TO_ENVIRONMENT}/utils /content
!cp -r {PATH_TO_ENVIRONMENT}/render_utils /content
!cp -r {PATH_TO_ENVIRONMENT}/entities /content
!cp -r {PATH_TO_ENVIRONMENT}/colliders /content
!cp {PATH_TO_ENVIRONMENT}/environment.py /content

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from A2C import AgentA2C
from colab_helpers import save_video, show_video
from environment import Environment

In [ ]:
# Path to save generated videos
VIDEO_FILE_PATH = '.'

# Mode in which environment is rendered
RENDER_MODE = 'rgb_array'

TIME_STEPS = 5000

prey_action_space = 4
prey_obs_space = 23

predator_action_space = 4
predator_obs_space = 17


prey_agent = AgentA2C(prey_action_space, prey_obs_space, units=256, dropout=0)
predator_agent = AgentA2C(predator_action_space, predator_obs_space, units=256, dropout=0)

prey_agent.load(PATH_TO_PREY_MODEL)
predator_agent.load(PATH_TO_PREDATOR_MODEL)

In [ ]:
env = Environment(render_mode=RENDER_MODE,
                  preys_initial_count=25,
                  plants_count=200,
                  predators_initial_count=25)

### Random movement of prey and predator


In [ ]:
frames = []
state = env.reset()
total_reward = 0

preys_over_time = []
predators_over_time = []
grown_plants_over_time = []

for time_step in range(TIME_STEPS):
  done = False

  for prey in env.get_preys():
    env.step_prey(prey, np.random.randint(0, prey_action_space))

  for predator in env.get_predators():
    env.step_predator(predator, np.random.randint(0, predator_action_space))

  env.step()

  frame = env.render()
  frames.append(frame)

  print(f"\rTime step: {time_step}, predators: {len(env.get_predators())}, preys: {len(env.get_preys())} , plants:{count}", end="")

  if len(env.get_preys()) == 0 and len(env.get_predators()) == 0:
    break
  else:
    preys_over_time.append(len(env.get_preys()))
    predators_over_time.append(len(env.get_predators()))

    count = 0
    for plant in env._plants:
      if plant._is_grown:
        count += 1

    grown_plants_over_time.append(count)

save_video(frames, f'{VIDEO_FILE_PATH}/Test of environment')
show_video(f'{VIDEO_FILE_PATH}/Test of environment.mp4')

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(range(len(preys_over_time)), preys_over_time, label="Ofiary", color='tab:blue', linewidth=3)
plt.plot(range(len(predators_over_time)), predators_over_time, label="Drapieżniki", color='tab:orange', linewidth=3)
plt.plot(range(len(grown_plants_over_time)), grown_plants_over_time, label="Rośliny", color='tab:green', linewidth=3)

plt.title("Dynamika populacji w czasie trwania symulacji", fontsize=16, fontweight='bold')
plt.xlabel("Krok czasowy", fontsize=14)
plt.ylabel("Liczba osobników", fontsize=14)

plt.legend(loc='upper right', fontsize=12)

plt.grid(True, linestyle='--', alpha=0.7)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
predator_statistics = env.get_predators_statistics()

drink_counts = [v['drink_count'] for v in predator_statistics.values()]
eat_counts = [v['eat_count'] for v in predator_statistics.values()]
reproduction_counts = [v['reproduction_count'] for v in predator_statistics.values()]
survived_time_steps = [v['survived_time_steps'] for v in predator_statistics.values()]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(drink_counts, color='tab:blue', edgecolor='black')
axes[0, 0].set_title("Histogram liczby nawodnień przy źródle wody", fontsize=16, fontweight='bold')
axes[0, 0].set_xlabel("Liczba nawodnień", fontsize=14)
axes[0, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 0].grid(True, linestyle='--', alpha=0.7)

axes[0, 1].hist(eat_counts, color='tab:green', edgecolor='black')
axes[0, 1].set_title("Histogram liczby posiłków", fontsize=16, fontweight='bold')
axes[0, 1].set_xlabel("Liczba posiłków", fontsize=14)
axes[0, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 1].grid(True, linestyle='--', alpha=0.7)

axes[1, 0].hist(reproduction_counts, color='tab:purple', edgecolor='black')
axes[1, 0].set_title("Histogram liczby reprodukcji", fontsize=16, fontweight='bold')
axes[1, 0].set_xlabel("Liczba reprodukcji", fontsize=14)
axes[1, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 0].grid(True, linestyle='--', alpha=0.7)

axes[1, 1].hist(survived_time_steps, color='tab:gray', edgecolor='black')
axes[1, 1].set_title("Histogram liczby przetrwanych kroków czasowych", fontsize=16, fontweight='bold')
axes[1, 1].set_xlabel("Kroki czasowe", fontsize=14)
axes[1, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 1].grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()

plt.show()

In [ ]:
prey_statistics = env.get_preys_statistics()

drink_counts = [v['drink_count'] for v in prey_statistics.values()]
eat_counts = [v['eat_count'] for v in prey_statistics.values()]
reproduction_counts = [v['reproduction_count'] for v in prey_statistics.values()]
survived_time_steps = [v['survived_time_steps'] for v in predator_statistics.values()]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(drink_counts, color='tab:blue', edgecolor='black')
axes[0, 0].set_title("Histogram liczby nawodnień przy źródle wody", fontsize=16, fontweight='bold')
axes[0, 0].set_xlabel("Liczba nawodnień", fontsize=14)
axes[0, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 0].grid(True, linestyle='--', alpha=0.7)

axes[0, 1].hist(eat_counts, color='tab:green', edgecolor='black')
axes[0, 1].set_title("Histogram liczby posiłków", fontsize=16, fontweight='bold')
axes[0, 1].set_xlabel("Liczba posiłków", fontsize=14)
axes[0, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 1].grid(True, linestyle='--', alpha=0.7)

axes[1, 0].hist(reproduction_counts, color='tab:purple', edgecolor='black')
axes[1, 0].set_title("Histogram liczby reprodukcji", fontsize=16, fontweight='bold')
axes[1, 0].set_xlabel("Liczba reprodukcji", fontsize=14)
axes[1, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 0].grid(True, linestyle='--', alpha=0.7)

axes[1, 1].hist(survived_time_steps, color='tab:gray', edgecolor='black')
axes[1, 1].set_title("Histogram liczby przetrwanych kroków czasowych", fontsize=16, fontweight='bold')
axes[1, 1].set_xlabel("Kroki czasowe", fontsize=14)
axes[1, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 1].grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()

plt.show()

In [ ]:
plants_statistics = env.get_plants_statistics()

values = list(plants_statistics.values())

plt.figure(figsize=(10, 4))
plt.hist(values, color='tab:blue', edgecolor='black')
plt.title("Histogram liczby zjedzeń roślin", fontsize=16, fontweight='bold')
plt.xlabel("Liczba zjedzeń", fontsize=14)
plt.ylabel("Częstotliwość", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

### Trained predator and prey environment

In [ ]:
env.reset()

frames = []

preys_obs_dict = {prey._id: env.obs_prey(prey) for prey in env.get_preys()}
predators_obs_dict = {predator._id: env.obs_predator(predator) for predator in env.get_predators()}

preys_over_time = []
predators_over_time = []
grown_plants_over_time = []

for time_step in range(TIME_STEPS):

  for prey in env.get_preys():
    if prey._id not in preys_obs_dict:
      preys_obs_dict[prey._id] =  env.obs_prey(prey)

    action = prey_agent.act(preys_obs_dict[prey._id])
    obs_prey, _, _ = env.step_prey(prey, action)
    preys_obs_dict[prey._id] = obs_prey


  for predator in env.get_predators():
    if predator._id not in predators_obs_dict:
      predators_obs_dict[predator._id] = env.obs_predator(predator)

    action_predator = predator_agent.act(predators_obs_dict[predator._id])
    obs_predator, _, _ = env.step_predator(predator, action_predator)
    predators_obs_dict[predator._id] = obs_predator

  env.step()

  frame = env.render()
  frames.append(frame)

  if len(env.get_preys()) == 0 and len(env.get_predators()) == 0:
    break
  else:
    preys_over_time.append(len(env.get_preys()))
    predators_over_time.append(len(env.get_predators()))

    count = 0
    for plant in env._plants:
      if plant._is_grown:
        count += 1

    grown_plants_over_time.append(count)

    print(f"\rTime step: {time_step}, predators: {len(env.get_predators())}, preys: {len(env.get_preys())} , plants:{count}", end="")

save_video(frames, f'{VIDEO_FILE_PATH}/Trained model')
show_video(f'{VIDEO_FILE_PATH}/Trained model.mp4')

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(range(len(preys_over_time)), preys_over_time, label="Ofiary", color='tab:blue', linewidth=3)
plt.plot(range(len(predators_over_time)), predators_over_time, label="Drapieżniki", color='tab:orange', linewidth=3)
plt.plot(range(len(grown_plants_over_time)), grown_plants_over_time, label="Rośliny", color='tab:green', linewidth=3)

plt.title("Dynamika populacji w czasie trwania symulacji", fontsize=16, fontweight='bold')
plt.xlabel("Krok czasowy", fontsize=14)
plt.ylabel("Liczba osobników", fontsize=14)

plt.legend(loc='upper right', fontsize=12)

plt.grid(True, linestyle='--', alpha=0.7)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
prey_statistics = env.get_preys_statistics()

drink_counts = [v['drink_count'] for v in prey_statistics.values()]
eat_counts = [v['eat_count'] for v in prey_statistics.values()]
reproduction_counts = [v['reproduction_count'] for v in prey_statistics.values()]
survived_time_steps = [v['survived_time_steps'] for v in prey_statistics.values()]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(drink_counts, color='tab:blue', edgecolor='black')
axes[0, 0].set_title("Histogram liczby nawodnień przy źródle wody", fontsize=16, fontweight='bold')
axes[0, 0].set_xlabel("Liczba nawodnień", fontsize=14)
axes[0, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 0].grid(True, linestyle='--', alpha=0.7)

axes[0, 1].hist(eat_counts, color='tab:green', edgecolor='black')
axes[0, 1].set_title("Histogram liczby posiłków", fontsize=16, fontweight='bold')
axes[0, 1].set_xlabel("Liczba posiłków", fontsize=14)
axes[0, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 1].grid(True, linestyle='--', alpha=0.7)

axes[1, 0].hist(reproduction_counts, color='tab:purple', edgecolor='black')
axes[1, 0].set_title("Histogram liczby reprodukcji", fontsize=16, fontweight='bold')
axes[1, 0].set_xlabel("Liczba reprodukcji", fontsize=14)
axes[1, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 0].grid(True, linestyle='--', alpha=0.7)

axes[1, 1].hist(survived_time_steps, color='tab:gray', edgecolor='black')
axes[1, 1].set_title("Histogram liczby przetrwanych kroków czasowych", fontsize=16, fontweight='bold')
axes[1, 1].set_xlabel("Kroki czasowe", fontsize=14)
axes[1, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 1].grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()

plt.show()

In [ ]:
predator_statistics = env.get_predators_statistics()

drink_counts = [v['drink_count'] for v in predator_statistics.values()]
eat_counts = [v['eat_count'] for v in predator_statistics.values()]
reproduction_counts = [v['reproduction_count'] for v in predator_statistics.values()]
survived_time_steps = [v['survived_time_steps'] for v in predator_statistics.values()]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(drink_counts, color='tab:blue', edgecolor='black')
axes[0, 0].set_title("Histogram liczby nawodnień przy źródle wody", fontsize=16, fontweight='bold')
axes[0, 0].set_xlabel("Liczba nawodnień", fontsize=14)
axes[0, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 0].grid(True, linestyle='--', alpha=0.7)

axes[0, 1].hist(eat_counts, color='tab:green', edgecolor='black')
axes[0, 1].set_title("Histogram liczby posiłków", fontsize=16, fontweight='bold')
axes[0, 1].set_xlabel("Liczba posiłków", fontsize=14)
axes[0, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[0, 1].grid(True, linestyle='--', alpha=0.7)

axes[1, 0].hist(reproduction_counts, color='tab:purple', edgecolor='black')
axes[1, 0].set_title("Histogram liczby reprodukcji", fontsize=16, fontweight='bold')
axes[1, 0].set_xlabel("Liczba reprodukcji", fontsize=14)
axes[1, 0].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 0].grid(True, linestyle='--', alpha=0.7)

axes[1, 1].hist(survived_time_steps, color='tab:gray', edgecolor='black')
axes[1, 1].set_title("Histogram liczby przetrwanych kroków czasowych", fontsize=16, fontweight='bold')
axes[1, 1].set_xlabel("Kroki czasowe", fontsize=14)
axes[1, 1].set_ylabel("Częstotliwość", fontsize=14)
axes[1, 1].grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()

plt.show()

In [ ]:
plants_statistics = env.get_plants_statistics()

values = list(plants_statistics.values())

plt.figure(figsize=(10, 4))
plt.hist(values, color='tab:blue', edgecolor='black')
plt.title("Histogram liczby zjedzeń roślin", fontsize=16, fontweight='bold')
plt.xlabel("Liczba zjedzeń", fontsize=14)
plt.ylabel("Częstotliwość", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()